In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Definir las constantes
g = 9.8 # gravedad [m/s^2]
p_atm = 101325 # Presión atmosférica [Pa]
beta = 3.67e-3 # Coeficiente de expansion termica [1/K]
T_w = 80.0 + 273.0 # Temperatura de la pared [K]
T_a = 20.0 + 273.0 # Temperatura ambiente [K]
delta_T = T_w - T_a
lambda_f = 0.04 # Coeficiente de friccion [adimensional]
R = 10.0 # Radio de la tuberia [m]
L = 100.0 # Longitud de la tuberia [m]
alpha = 2.2e-5 # Coeficiente de difusividad [m^2/s]
rho_0 = 1.0 # Densidad [kg/m^3]
Cp = 1.012 # Capacidad calorifica [J/(kg K)]
k = 0.024 # Conductividad termica [W/(m K)]
nu = 1.71e-5 # Viscosidad [Pa s]
Ra_D = (g * beta * delta_T * (2 * R)**3)/ nu**2 # Numero de Rayleigh
Nu = (576/(Ra_D * 2 * R /L)**2 + 2.873/(Ra_D * 2 * R /L)**(1/2))**(-1/2)# Numero de Nusselt
h_v = (k * Nu)/ (2 * R * L) # coeficiente de tranferencia de calor por convección [W/(m^3 K)]

## Solución estacionaria, calculo de $u_e$

In [9]:
def f(u, A, B):
    return u - A * (1 - np.exp(-B/u))

def df(u, A, B):
    return 1 + A * (B/u**2) * np.exp(-B/u)

def newton_raphson(A, B, u0, tol, max_iter):
    u = u0
    for i in range(max_iter):
        f_val = f(u, A, B)
        df_val = df(u, A, B)
        u_new = u - f_val / df_val
        if np.abs(u_new - u) < tol:
            return u_new
        u = u_new
    return u

# Example usage
A = rho_0 * Cp * g * beta * (T_w - T_a) / (h_v * (0.5 + lambda_f * L /(8 * R)))
B = h_v * L / (rho_0 * Cp)
u0 = 0.01
tol = 1e-6
max_iter = 100

u_sol = newton_raphson(A, B, u0, tol, max_iter)
u_e = u_sol
print('u_e = ' + str(round(u_e,4))+ ' m/s')

u_e = 19.4829 m/s


## Calculo la temperatura final para poder escalar la temperatura del aire entre 0 y 1.
## + def de parametros adimensionales

In [12]:
gamma = h_v * L / (rho_0 * Cp * u_e)
T_f = T_a + (T_w - T_a) * (1 - np.exp(-gamma))
print('T_f = ' + str(round(T_f -273, 4)) + 'ºC')

Pi_d = rho_0 * u_e**2 / p_atm
Pi_0 = rho_0 * g * L / p_atm
phi = L * g * beta * (T_f - T_a) / u_e**2
theta_r = (T_w -T_a) / (T_f - T_a)
LAMBDA = lambda_f * L / (8 * R)
delta = alpha /(u_e * L)


T_f = 23.8637ºC


## Simulación numérica

In [23]:
# Definición de variables y condiciones iniciales
N = 100 # Número de puntos en la dirección temporal
M = 200 # Número de puntos en la dirección vertical
eta = np.linspace(0, 1, M)  # Coordenada vertical adimensional
tau = np.linspace(0, 1, N)  # Coordenada temporal adimensional
deta = (eta[-1] - eta[0]) / M
dtau = (tau[-1] - tau[0]) / N
v = np.zeros(N)  # Velocidad adimensional
theta = np.zeros((N, M))  # Temperatura adimensional
Pi = np.zeros((N, M))  # Presión adimensional

# Condiciones iniciales
v[0] = 0.0  # Velocidad inicial cero
theta[0, :] = 0.0  # Temperatura inicial uniforme
Pi[0, :] = 1.0  # Presión inicial uniforme
Pi[:,-1] = 1 # La presión en todos los instantes en la posición final será la misma (P_atm)
 
# Simulación numérica
for i in range(N-1):

    # Calculo de la velocidad en el instante i+1 a partir de la presión en eta = 0 del instante anterior:
    v[i+1] = np.sqrt(2 * (1 - Pi[i, 0]))

    # Calculo de las distribuciones de temperatura y presion en el espacio para el instante i+1:
    for j in range(M-1):
         
        theta[i+1, j] = theta[i, j] + dtau * (- v[i+1] * (theta[i, j+1] - theta[i, j]) / deta
                                                              + delta * (theta[i, j-1] - 2 * theta[i, j] + theta[i, j+1]) / deta**2
                                                              - gamma * (theta[i, j] - theta_r))
        
        Pi[i+1, M-2-j] = Pi[i+1, M-1-j] + deta * Pi_d * ((v[i+1] - v[i]) / dtau + phi * (theta[i+1, M-2-j] - theta_r) - LAMBDA * v[i+1]**2)
         

print(Pi)
print(v)
print(theta_r)
print(LAMBDA)
print(Pi_d)
print(deta)
print(dtau)


[[1.         1.         1.         ... 1.         1.         1.        ]
 [0.99788162 0.99789226 0.9979029  ... 0.9999787  0.99998935 1.        ]
 [1.02214378 1.0220325  1.02192122 ... 1.00022254 1.00011127 1.        ]
 ...
 [       nan        nan        nan ...        nan        nan 1.        ]
 [       nan        nan        nan ...        nan        nan 1.        ]
 [       nan        nan        nan ...        nan        nan 1.        ]]
[0.         0.         0.06509045        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        n

/var/folders/lx/vhxvbgp94fj63r8wrldf_13m0000gn/T/ipykernel_1799/1163777213.py:22: RuntimeWarning: invalid value encountered in sqrt
  v[i+1] = np.sqrt(2 * (1 - Pi[i, 0]))


In [24]:
print(Pi[:,0])

[1.         0.99788162 1.02214378        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan       